In [53]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.patches as patches
import mediapipe as mp 

mp_face_detection = mp.solutions.face_detection 

def detect_face_mp(image):
    with mp_face_detection.FaceDetection(min_detection_confidence=0.1) as face_detection:
        # Convert to RGB
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Process and get face detections
        results = face_detection.process(rgb_image)
        faces = []
        
        if results.detections:
            for detection in results.detections:
                bboxC = detection.location_data.relative_bounding_box
                ih, iw, _ = image.shape
                x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                faces.append([x, y, w, h])
        return faces



import face_recognition

def detect_face_fc(image):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB as face_recognition expects RGB images
    face_locations = face_recognition.face_locations(rgb_image)

    faces = []
    for (top, right, bottom, left) in face_locations:
        faces.append([left, top, right - left, bottom - top])
        
    return faces


def detect_face_cv(image):
    net = cv2.dnn.readNetFromCaffe("models/face/deploy.prototxt", "models/face/res10_300x300_ssd_iter_140000.caffemodel")
    h, w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 1.0, (300, 300), (104.0, 177.0, 123.0))
    
    net.setInput(blob)
    detections = net.forward()
    
    faces = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # you can adjust this threshold
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            faces.append([startX, startY, endX-startX, endY-startY])

    return faces


def apply_transformations(depth_image):
    depth_image = np.nan_to_num(depth_image, nan=0.0)  # Replace NaNs with 0
    depth_image[depth_image > 1.3] = 0  # Thresholding
    
    non_zero_indices = np.nonzero(depth_image)
    if non_zero_indices[0].size > 0:
        min_val = np.min(depth_image[non_zero_indices])  # Ignore zeros
        max_val = np.max(depth_image)
        
        depth_image[non_zero_indices] = (depth_image[non_zero_indices] - min_val) / (max_val - min_val)  # Normalization
    
    depth_image = np.power(depth_image, 1.5)  # Gamma correction
    
    depth_image = 1 - depth_image  # Invert the depth image so that less depth appears lighter

    return depth_image


def find_max_indices(folder_path):
    max_x_index = max_y_index = -1

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.npy'):
                depth_data = np.load(os.path.join(root, file))
                cur_max_x_index = depth_data.shape[1] - 1
                cur_max_y_index = depth_data.shape[0] - 1
                max_x_index = max(max_x_index, cur_max_x_index)
                max_y_index = max(max_y_index, cur_max_y_index)

    return max_x_index, max_y_index

def pad_depth_arrays(input_folder, output_folder, max_x_index, max_y_index):
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.npy'):
                depth_data = np.load(os.path.join(root, file))
                depth_data[np.isnan(depth_data)] = 0 # convert nan values to 0
                depth_data[(depth_data > 1.4)] = 0   # Applying threshold
                target_shape = (158, 155)

                padded_depth_data = np.pad(
                    depth_data,
                    ((0, target_shape[0] - depth_data.shape[0]), (0, target_shape[1] - depth_data.shape[1])),
                    mode='constant', constant_values=0
                )

                rel_path = os.path.relpath(root, input_folder)
                output_subfolder = os.path.join(output_folder, rel_path)
                
                try:
                    os.makedirs(output_subfolder, exist_ok=True)
                    print(f"Successfully created the directory {output_subfolder}")
                except Exception as e:
                    print(f"Failed to create directory. Error: {e}")

                output_file = os.path.join(output_subfolder, file)
                np.save(output_file, padded_depth_data)


def read_txt_to_depth_image_fd(file_path, width, height):
    depth_image = np.zeros((height, width))
    with open(file_path, 'r') as f:
        next(f)  # Skip header line
        for line in f:
            i, j, x, y, z, _, _ = map(float, line.strip().split('\t'))
            if 0 <= int(i) < width and 0 <= int(j) < height:
                depth_image[int(j), int(i)] = z
    return depth_image



def read_txt_to_depth_image_tr(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()[1:]  # Skip the header line
        depth_image = np.loadtxt(lines)
    return depth_image


def read_txt_to_depth_array_np(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()[1:]  # Skip the header line
        depth_data = np.loadtxt(lines)
    return depth_data


def perform_first_code_actions(person_dir, final_depth_folder):
    for filename in os.listdir(person_dir):
        if filename.endswith('_data.txt'):
            person, file_number = map(int, filename.split('_')[:2])
            print(f"Processing person {person} file number {file_number}")
            input_text_path = os.path.join(person_dir, filename)
            depth_data = read_txt_to_depth_array_np(input_text_path)
            if not os.path.exists(final_depth_folder):
                os.makedirs(final_depth_folder)
            output_data_path = os.path.join(final_depth_folder, f"{str(person).zfill(3)}_{str(file_number).zfill(2)}_depth_data.npy")
            np.save(output_data_path, depth_data)


def print_padded_arrays(output_folder):
    count = 0
    for root, _, files in os.walk(output_folder):
        for file in files:
            if file.endswith('.npy'):
                padded_depth_data = np.load(os.path.join(root, file))
                print(f'Array from File: {os.path.join(root, file)}')
                print(f'Shape: {padded_depth_data.shape}')
                count += 1
    print(count)



# Mapping RGB facial data onto Depth Data
### Attempt face detection using libraries : face_recongiton -> Mediapipe -> OpenCV

In [44]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Assuming detect_face_fc() and detect_face_mp() are defined elsewhere


my_test = "test5"

depth_width, depth_height = 960, 540
rgb_width, rgb_height = 1920, 1080

depth_folder = f'test_data/{my_test}_output'
people_dirs = [os.path.join(depth_folder, d) for d in os.listdir(depth_folder) if os.path.isdir(os.path.join(depth_folder, d))]

for person_dir in people_dirs:
    person = os.path.basename(person_dir)
    depth_files = [f for f in os.listdir(person_dir) if f.endswith('_cloud.txt')]
    
    for depth_file in depth_files:
        file_number_str = depth_file.split('_')[1]
        file_number = int(file_number_str)

        print(f"person: {person}  image: {file_number}")

        depth_file_path = os.path.join(person_dir, depth_file)
        rgb_file_path = f'test_data/{my_test}_output/{person}/{person}_{str(file_number).zfill(2)}_image.png'
        
        print("depth path: " + str(depth_file_path) + " rgb_file_path: " + str(rgb_file_path))

        if not os.path.exists(depth_file_path) or not os.path.exists(rgb_file_path):
            continue

        depth_image = read_txt_to_depth_image_fd(depth_file_path, depth_width, depth_height)
        rgb_image = cv2.imread(rgb_file_path)

        faces = detect_face_fc(rgb_image)
        if len(faces) == 0:
            faces = detect_face_mp(rgb_image)
            if len(faces) == 0:
                continue

        for (x, y, w, h) in faces:
            cropped_rgb_face = rgb_image[y:y+h, x:x+w]

            # Save cropped RGB face image
            output_rgb_dir = f'test_data/{my_test}_face_result/{person}'
            if not os.path.exists(output_rgb_dir):
                os.makedirs(output_rgb_dir)
            cv2.imwrite(f"{output_rgb_dir}/{person}_{str(file_number).zfill(2)}_rgb.png", cropped_rgb_face)
        
        scale_x, scale_y = depth_width / rgb_width, depth_height / rgb_height
        x, y, w, h = int(x * scale_x), int(y * scale_y), int(w * scale_x), int(h * scale_y)
        face_region_depth = depth_image[y:y+h, x:x+w]

        output_dir = f'test_data/{my_test}_face_result/{person}'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        output_text_path = f"{output_dir}/{str(person).zfill(3)}_{str(file_number).zfill(2)}_data.txt"

        # Save the text file with resized values
        with open(output_text_path, 'w') as f:
            f.write("z-values for each (x, y) in a 2D grid\n")
            for j in range(face_region_depth.shape[0]):  # Vertical (y-axis)
                for i in range(face_region_depth.shape[1]):  # Horizontal (x-axis)
                    z_value = face_region_depth[j, i]
                    f.write(f"{z_value}\t")
                f.write("\n")
        
        output_depth_face = f'test_data/{my_test}_face_result/{person}'
        if not os.path.exists(output_depth_face):
            os.makedirs(output_depth_face)
        #plt.imsave(f"{output_depth_face}/{str(person).zfill(3)}_{str(file_number).zfill(2)}_depth.png", face_region_depth, cmap='gray')

        depth_image = read_txt_to_depth_image_tr(output_text_path)
        transformed_depth_image = apply_transformations(depth_image)
        
        # Save the transformed depth image
        output_image_path = os.path.join(output_depth_face, f"{str(person).zfill(3)}_{str(file_number).zfill(2)}_depth.png")
        plt.imsave(output_image_path, transformed_depth_image, cmap='gray')


person: 007  image: 8
depth path: test_data/test5_output/007/007_08_cloud.txt rgb_file_path: test_data/test5_output/007/007_08_image.png
person: 010  image: 8
depth path: test_data/test5_output/010/010_08_cloud.txt rgb_file_path: test_data/test5_output/010/010_08_image.png
person: 018  image: 8
depth path: test_data/test5_output/018/018_08_cloud.txt rgb_file_path: test_data/test5_output/018/018_08_image.png
person: 017  image: 8
depth path: test_data/test5_output/017/017_08_cloud.txt rgb_file_path: test_data/test5_output/017/017_08_image.png
person: 001  image: 8
depth path: test_data/test5_output/001/001_08_cloud.txt rgb_file_path: test_data/test5_output/001/001_08_image.png
person: 003  image: 8
depth path: test_data/test5_output/003/003_08_cloud.txt rgb_file_path: test_data/test5_output/003/003_08_image.png
person: 005  image: 8
depth path: test_data/test5_output/005/005_08_cloud.txt rgb_file_path: test_data/test5_output/005/005_08_image.png
person: 013  image: 8
depth path: test_da

<div class="alert alert-block alert-info">
Check face detection results in results in <b>face_result</b> folder.
<p>If files are missing use <b>"bouding_box.py"</b> to manually select face region for a specific file.<b>(Recommended)</b></p>
<p>You can also use <b>"face_detector_fix.py"</b> to test out different face detectors.</p>
</div>

## Convert to Numpy Arrays, add Padding, and apply Threshold
#### Transform (y, x) => (158, 155)
#### Threshold => Depth values > 1.3 = 0 

In [54]:
import os
import numpy as np


use_padding = True

my_test = "test5"
root_folder = f'test_data/{my_test}_face_result'
input_folder = os.path.join('test_data', f'{my_test}_depth_numpy')
print(input_folder)


if use_padding:
    output_folder = os.path.join('test_data', f'{my_test}_padded_depth_numpy')
else:
    output_folder = os.path.join('test_data', f'{my_test}_depth_numpy')

people_dirs = [os.path.join(root_folder, d) for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))]

for person_dir in people_dirs:
    person = os.path.basename(person_dir)
    final_depth_folder = os.path.join('test_data', f"{my_test}_depth_numpy", str(person))
    perform_first_code_actions(person_dir, final_depth_folder)

if use_padding:
    max_x_index, max_y_index = find_max_indices(input_folder)
    pad_depth_arrays(input_folder, output_folder, max_x_index, max_y_index)


print_padded_arrays(output_folder)


test_data/test5_depth_numpy
Processing person 8 file number 8
Processing person 6 file number 8
Processing person 4 file number 8
Successfully created the directory test_data/test5_padded_depth_numpy/008
Successfully created the directory test_data/test5_padded_depth_numpy/006
Successfully created the directory test_data/test5_padded_depth_numpy/004
Array from File: test_data/test5_padded_depth_numpy/008/008_08_depth_data.npy
Shape: (158, 155)
Array from File: test_data/test5_padded_depth_numpy/006/006_08_depth_data.npy
Shape: (158, 155)
Array from File: test_data/test5_padded_depth_numpy/004/004_08_depth_data.npy
Shape: (158, 155)
3


<div class="alert alert-block alert-success">
<h3>Your data is now for training/testing and is stored in <b>__padded_depth_numpy</b></h3>
<h5>You can now move to <b>train_test.ipynb</b>
</div>